In [1]:
###背景：在transformer架构中，input作为transformer中输入的起点，任意的字符串作为输入，经由分词器，将字符串分隔成具有不同语义的字符串数组，然后使用对应的embedding模型对字符串数组进行潜入操作，从而生成具有不同语义的向量数组。那这个embedding到底做了什么，为什么要embedding，embedding后的内容是什么呢？
###今天我们就针对aigc中input输入内容embedding过程进行分析，从而实现以下目的：
###1. embedding 是什么？ 
###2. 为什么要embedding？
###3. embedding 模型的选型
###4. 如何使用pytouch构建embedding

### Embedding是什么？
Embedding 是一种将离散的、高维的、没有语义关联的符号数据（如单词或类别标签）映射到低维的、具有语义关联的连续向量空间的技术。这个连续向量表示被称为嵌入（Embedding）。嵌入的目的是为了让计算机更好地理解和处理这些符号数据，同时减少计算复杂度。

### 为什么要使用Embedding?
使用Embedding的原因有两点：
#### 减少维度和计算复杂度：
将高维的离散数据转换为低维的连续向量可以减少计算和存储资源的需求，特别是在处理大型词汇表时。相比于独热编码等高维表示，嵌入向量的维度通常较低，但仍能保留数据的语义信息。
#### 引入语义关联：
嵌入向量的另一个重要特性是，相似的符号在嵌入空间中会更接近。这意味着具有相似语义的单词或类别标签在向量空间中距离较近，从而可以更好地捕捉它们之间的语义关系。这对于自然语言处理等任务非常有用，因为模型可以更好地泛化和理解文本数据。

总之，Embedding 是一种将离散数据转换为连续向量的技术，它旨在减少维度和引入语义关联，以帮助计算机更好地处理符号数据。您的解释大体正确，只是稍微添加了一些澄清和补充信息。

### Embedding历程
#### 词汇表和One-Hot Encoding
在早期的计算机科学和自然语言处理中，为了处理文本数据，人们通常会建立一个词汇表，其中包含了语言中的所有词汇。计算机通过将每个单词映射到词汇表中的一个索引或位置，以将文本表示为数字形式。One-Hot Encoding（独热编码）是一种常见的方法，它将每个单词表示为一个高维度的二进制向量，其中只有一个位置是1，其余都是0。这个方法容易理解，但导致了高维度的表示，浪费了存储空间并增加了计算复杂性。

#### 嵌入（Embedding）的引入
随着深度学习和神经网络的发展，科学家们引入了嵌入技术，这是一种将离散的符号数据映射到低维度的连续向量空间的方法。嵌入矩阵被用来将每个单词或符号映射到一个向量，而这些向量之间可以具有语义关联。这种方法在自然语言处理中取得了巨大的成功，因为它更有效地表示了语言的语义信息。在嵌入空间中，如果两个向量在某个维度上靠近，那么表示这两个符号的词也在语义上相似。

嵌入技术的出现确实是一项重要的进步，使得计算机能够更好地理解和处理自然语言，同时减少了维度和提高了语义关联性


### 常见的embedding 模型的选型

### 构建embedding
PyTorch是一个深度学习框架，借助于该框架提供的工具和算法，我们可以基于大量数据和特定算法训练出针对特定场景的模型。在下文中，我们使用PyTorch的nn模块来构建嵌入层，这是一种通用的构建嵌入层的方法。而像Word2Vec等模型/算法，则是在经过大量数据和专门的算法训练后得到的。PyTorch构建的嵌入层经过特定算法和大量数据的训练后，也可以生成类似于Word2Vec等模型的嵌入

In [2]:
#安装依赖
!pip3 install -U PyTouch

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/8f/47/33b9d0df325b42acd1081dbcca46623d5175d49713e75b48884155879fd9/pytouch-0.4.2-py3-none-any.whl (66 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
import matplotlib.pyplot as plt
import numpy as np
import copy

model_dimension = 1536 #表示潜入模型的维度，像open ai的embedding-002是1536维
vocab_size = 1000 #表示词汇表的大小，一般是表示这个可能出现的词的多少

#构建embedding 对象
#Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None)
#num_embeddings 表示符号数据的数量，比如您正在处理文本的潜入层，此时该参数表示词汇表的大小
#embedding-dim 表示潜入的维度，比如open-ai 的embedding-002，该参数的值是1536
embedding = nn.Embedding(vocab_size, model_dimension)

#构建需要处理的数据，一般是以NxN维的方式提供，即一个[n][n]的数组
# PyTorch 张量，是PyTorch的核心数据结构之一，他类似于多维数组或矩阵，非常适用于处理如图像、文本、序列等
# 优势如下：
# 数值计算：PyTorch张量是为数值计算而优化的，可以执行各种数学运算，如加法、乘法、矩阵乘法、梯度计算等。这使得PyTorch成为深度学习任务的理想选择。
# 自动求导：PyTorch的张量具有自动求导功能，也就是说，您可以在张量上执行操作，并且PyTorch会自动跟踪这些操作以计算梯度。这对于训练神经网络等需要反向传播的任务非常有用。
# GPU加速：PyTorch支持在GPU上执行张量计算，这可以显著提高深度学习模型的训练速度。您可以轻松地将张量从CPU转移到GPU上进行计算。
# 灵活性：PyTorch张量具有丰富的方法和函数，用于操作、索引、切片、连接、重塑和变换张量数据。这使得在深度学习任务中处理各种数据变得非常灵活。
# 深度学习框架集成：PyTorch张量是PyTorch深度学习框架的核心组件之一，可以与其他PyTorch模块（如神经网络层、优化器等）无缝集成。
# 丰富的功能：PyTorch张量支持各种功能，如广播（broadcasting）、张量拼接（tensor concatenation）、索引选择（indexing and selection）、元素级运算（element-wise operations）等。

#LongTensor 在 PyTorch 中主要用于存储整数值，通常用于表示分类标签、索引序列和符号数据的整数表示。这使得它在处理分类、自然语言处理和序列数据等任务时非常有用。
input = torch.LongTensor([[0, 2, 0, 5]])
print(input)
print(embedding(input))

tensor([[0, 2, 0, 5]])
tensor([[[ 0.5329, -0.7547, -0.5999,  ...,  1.3890, -0.8328,  1.0597],
         [-0.4141, -0.8679, -0.5504,  ...,  0.7097, -0.6024,  0.2964],
         [ 0.5329, -0.7547, -0.5999,  ...,  1.3890, -0.8328,  1.0597],
         [-0.9329,  1.7162,  0.1181,  ..., -0.5675, -0.0467, -0.5132]]],
       grad_fn=<EmbeddingBackward0>)
